In [1]:
from pytube import YouTube
from moviepy.editor import *
import os
import eyed3
from urllib import request

In [19]:
#creating video class using code that I developed in the audio scraper notebook
class video:
    
    def __init__(self, url):
        
        self.yt = YouTube(url)

        #storing track details for later
        self.track_title = self.yt.title
        self.uploader = self.yt.author

        #grabbing image for album cover

        self.thumbnail = request.urlopen(self.yt.thumbnail_url)
        self.cover = self.thumbnail.read()
    

RegexMatchError: __init__: could not find match for ^\w+\W